# Generacion Model Relacional

In [5]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

print("Abriendo ventana de selección...")

root = tk.Tk()
root.withdraw() 

ruta_archivo = filedialog.askopenfilename(
    title="Selecciona tu Excel",
    filetypes=[("Archivos Excel", "*.xlsx *.xls")]
)

if ruta_archivo:
    print(f"🚀 Cargando archivo: {ruta_archivo}")
    
    try:
        # PANDAS PURO: Esto debería tomar menos de 5 segundos
        # header=1: Asumimos que los títulos (MOVIL, RUC...) están en la fila 2 de Excel
        df_facturas = pd.read_excel(
            ruta_archivo, 
            sheet_name='2025', 
            header=5
        )
        
        print(f"✅ Lectura inicial completada. Filas detectadas: {len(df_facturas)}")

        # --- LIMPIEZA AUTOMÁTICA ---
        
        # 1. Eliminar columnas basura (las que se llaman "Unnamed: ...")
        #df_facturas = df_facturas.loc[:, ~df_facturas.columns.str.contains('^Unnamed')]

        # 2. Eliminar las "tablas pequeñitas" del final
        # Lógica: Si la columna 'MOVIL' (o 'RUC') está vacía, esa fila no sirve.
        # Ajusta 'MOVIL' por el nombre exacto de una columna que SIEMPRE deba tener datos.
        columna_clave = 'MOVIL' 
        
        if columna_clave in df_facturas.columns:
            filas_antes = len(df_facturas)
            df_facturas = df_facturas.dropna(subset=[columna_clave])
            filas_despues = len(df_facturas)
            print(f"🧹 Se eliminaron {filas_antes - filas_despues} filas (tablas extra al final o vacías).")
        
        # 3. Limpiar símbolos de moneda (S/) para poder sumar
        # Buscamos columnas que tengan texto tipo "S/" y las convertimos a números
        cols_monetarias = [c for c in df_facturas.columns if 'MONTO' in str(c).upper() or 'TOTAL' in str(c).upper()]
        
        for col in cols_monetarias:
            # Si la columna es de tipo texto (object), intentamos limpiarla
            if df_facturas[col].dtype == 'object':
                df_facturas[col] = (
                    df_facturas[col]
                    .astype(str)                  # Convertir a texto
                    .str.replace('S/', '')        # Quitar S/
                    .str.replace(',', '')         # Quitar comas de miles (si hay)
                    .str.replace(' ', '')         # Quitar espacios
                )
                # Convertir a número (los errores se vuelven NaN)
                df_facturas[col] = pd.to_numeric(df_facturas[col], errors='coerce')

        print("\n📊 DATOS LISTOS:")
        print(df_facturas.head())
        print(df_facturas.info())
        
    except Exception as e:
        print(f"❌ Error: {e}")

else:
    print("No seleccionaste archivo.")

Abriendo ventana de selección...
🚀 Cargando archivo: C:/Users/Usuario/Desktop/Scripts_automatización/Validacion_Facturacion_Liquidacion/2025 FACTURACIÓN Y COBRANZA.xlsx
✅ Lectura inicial completada. Filas detectadas: 5000

📊 DATOS LISTOS:
  Nombre Facturador Nombre de Facturador Ultimo TIPO DE FACTURACION  \
0             DANNA                       SOFIA           QUINCENAL   
1             DANNA                      ANDREE           QUINCENAL   
2             DANNA                       SOFIA             SEMANAL   
3            ANDREA                      ANDREA           QUINCENAL   
4             DANNA                   ALEJANDRA           QUINCENAL   

  Comprobante Relacionada       MES      NOMBRE EMPRESA O GRUPO  \
0          FA    F008-728  01.ENERO  RIMAC SEGUROS Y REASEGUROS   
1          FA           -  01.ENERO    CERVECERIA SAN JUAN S.A.   
2          FA    F008-754  01.ENERO            SAFRESCO PERU SA   
3          FA           -  01.ENERO                     ROTAPEL   

C:\Users\Usuario\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [6]:
df_facturas.head()

,Nombre Facturador,Nombre de Facturador Ultimo,TIPO DE FACTURACION,Comprobante,Relacionada,MES,NOMBRE EMPRESA O GRUPO,Nombre Empresa,CATEGORIA,RUC,...,BANCO,TIPO DE PAGO,TIPO DE NEGOCIACION,OBSERVACIONES,IMPORTE NETO Y PAGADO,PAGO DETRACCION,FECHA DE PAGO,PAGO RETENCION,FECHA DE PAGO2,/
0,DANNA,SOFIA,QUINCENAL,FA,F008-728,01.ENERO,RIMAC SEGUROS Y REASEGUROS,RIMAC SEGUROS Y REASEGUROS,NaN,20100041953,...,BCP-082,TRANSFERENCIA BANCARIA,transferencia,NaN,283.436,0.0,NaN,NaN,NaN,NaN
1,DANNA,ANDREE,QUINCENAL,FA,-,01.ENERO,CERVECERIA SAN JUAN S.A.,CERVECERIAS PERUANAS BACKUS SA,CERVECERIA SAN JUAN S.A.,20100113610,...,NaN,NaN,Factoring,NaN,93.200,0.0,NaN,NaN,NaN,NaN
2,DANNA,SOFIA,SEMANAL,FA,F008-754,01.ENERO,SAFRESCO PERU SA,SAFRESCO PERU SA,NaN,20136222725,...,BBVA-7571,PAGO RECURRENTE,Pago recurrente,NaN,70.800,0.0,NaN,NaN,NaN,NaN
3,ANDREA,ANDREA,QUINCENAL,FA,-,01.ENERO,ROTAPEL,ROTAPEL,NaN,20101314724,...,BCP-082,TRANSFERENCIA BANCARIA,transferencia,CLIENTE PAGA S/1 MAS DE DETRACCION,148.554,17.0,2025-08-01 00:00:00,NaN,NaN,NaN
4,DANNA,ALEJANDRA,QUINCENAL,FA,-,01.ENERO,MAPED PERU S.A.C.,MAPED PERU S.A.C.,NaN,20544563441,...,BBVA-7571,PAGO RECURRENTE,Pago recurrente,SE REALIZO AUTODETRACCION,1602.502,178.0,2025-08-13 00:00:00,NaN,NaN,NaN


In [7]:
df_facturas.columns

Index([                                 'Nombre Facturador',
                              'Nombre de Facturador Ultimo',
                                      'TIPO DE FACTURACION',
                                              'Comprobante',
                                              'Relacionada',
                                                      'MES',
                                   'NOMBRE EMPRESA O GRUPO',
                                           'Nombre Empresa',
                                                'CATEGORIA',
                                                      'RUC',
                                                  'Periodo',
                                          'Periodo Reporte',
                                              'PRIMER VALE',
                                              'ULTIMO VALE',
                                            'MONTO SIN IGV',
                                                      'IGV',
                        

In [8]:
# Definimos el mapeo de nombres (Excel -> SQL/Limpio)
mapeo_columnas = {
    'Nombre Facturador': 'nombre_facturador',
    'Nombre de Facturador Ultimo': 'nombre_facturador_ultimo',
    'TIPO DE FACTURACION': 'tipo_facturacion',
    'Comprobante': 'comprobante',
    'Relacionada': 'relacionada',
    'MES': 'mes_contable',
    'NOMBRE EMPRESA O GRUPO': 'empresa_grupo',
    'Nombre Empresa': 'nombre_empresa',
    'CATEGORIA': 'categoria',
    'RUC': 'ruc_cliente',
    'Periodo': 'periodo_operativo',
    'Periodo Reporte': 'periodo_reporte',
    'PRIMER VALE': 'primer_vale',
    'ULTIMO VALE': 'ultimo_vale',
    'MONTO SIN IGV': 'monto_sin_igv',
    'IGV': 'igv',
    '0.18': 'tasa_igv',  # Manejamos el caso donde el nombre es un número
    'MONTO FINAL ': 'monto_final', # Nota el espacio al final que venía en tu Index
    '# Servicios / Vales': 'cantidad_servicios',
    'Fecha de envío de Reporte': 'fecha_envio_reporte',
    'VB / FALTA': 'estado_vb',
    'Fecha Facturación': 'fecha_facturacion',
    'Fecha de recepcion': 'fecha_recepcion',
    '¿VALIDADO EN EL PORTAL?': 'validado_portal',
    'Vencimiento': 'fecha_vencimiento',
    'FECHA DE PAGO': 'fecha_pago_1',
    'PAGO RETENCION': 'pago_retencion',
    'FECHA DE PAGO2': 'fecha_pago_2',
    '/': 'observacion_slash'
}


df_facturas = df_facturas.rename(columns=mapeo_columnas)


df_facturas.columns = df_facturas.columns.str.strip()

df_facturas.head()

,nombre_facturador,nombre_facturador_ultimo,tipo_facturacion,comprobante,relacionada,mes_contable,empresa_grupo,nombre_empresa,categoria,ruc_cliente,...,BANCO,TIPO DE PAGO,TIPO DE NEGOCIACION,OBSERVACIONES,IMPORTE NETO Y PAGADO,PAGO DETRACCION,fecha_pago_1,pago_retencion,fecha_pago_2,observacion_slash
0,DANNA,SOFIA,QUINCENAL,FA,F008-728,01.ENERO,RIMAC SEGUROS Y REASEGUROS,RIMAC SEGUROS Y REASEGUROS,NaN,20100041953,...,BCP-082,TRANSFERENCIA BANCARIA,transferencia,NaN,283.436,0.0,NaN,NaN,NaN,NaN
1,DANNA,ANDREE,QUINCENAL,FA,-,01.ENERO,CERVECERIA SAN JUAN S.A.,CERVECERIAS PERUANAS BACKUS SA,CERVECERIA SAN JUAN S.A.,20100113610,...,NaN,NaN,Factoring,NaN,93.200,0.0,NaN,NaN,NaN,NaN
2,DANNA,SOFIA,SEMANAL,FA,F008-754,01.ENERO,SAFRESCO PERU SA,SAFRESCO PERU SA,NaN,20136222725,...,BBVA-7571,PAGO RECURRENTE,Pago recurrente,NaN,70.800,0.0,NaN,NaN,NaN,NaN
3,ANDREA,ANDREA,QUINCENAL,FA,-,01.ENERO,ROTAPEL,ROTAPEL,NaN,20101314724,...,BCP-082,TRANSFERENCIA BANCARIA,transferencia,CLIENTE PAGA S/1 MAS DE DETRACCION,148.554,17.0,2025-08-01 00:00:00,NaN,NaN,NaN
4,DANNA,ALEJANDRA,QUINCENAL,FA,-,01.ENERO,MAPED PERU S.A.C.,MAPED PERU S.A.C.,NaN,20544563441,...,BBVA-7571,PAGO RECURRENTE,Pago recurrente,SE REALIZO AUTODETRACCION,1602.502,178.0,2025-08-13 00:00:00,NaN,NaN,NaN


In [9]:
len(df_facturas)

5000

In [11]:
df_facturas['primer_vale'].head()

0   2025-01-06
1   2025-01-21
2   2025-01-21
3   2025-01-07
4   2025-01-01
Name: primer_vale, dtype: datetime64[ns]

In [ ]:
import pandas as pd
import pypyodbc as odbc
from sqlalchemy import create_engine, types
from sqlalchemy.engine import URL, create_engine

# Parámetros de conexión (guardarlo)
servername = 'LP-MP2P84ND-PE'
database = 'Facturacion'
username = 'sa'
password = '@nalistaDirecto'
# El driver (verificar si es el más actual)
driver = 'ODBC Driver 17 for SQL Server' 

# Creación de la URL de conexión para SQLAlchemy
connection_string = f"DRIVER={{{driver}}};SERVER={servername};DATABASE={database};UID={username};PWD={password}"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

engine = create_engine(connection_url, fast_executemany=True)


In [16]:
#Tablas a cargar
tb_data = {
    'tb_facturacion': df_facturas
}

In [19]:
from sqlalchemy import create_engine
import urllib


def cargar_multiples_tablas_seguro(tb_data, engine):
    for nombre_tabla, df in tb_data.items():
        try:
            print(f"🚀 Iniciando carga optimizada de {nombre_tabla}...")
            
            # Configuración de conexión e interación con BD
            with engine.connect() as conn:
                # Considerar CHUNKSIZE para carga masiva de lo contrario demora
                df.to_sql(
                    nombre_tabla, 
                    con=conn, 
                    if_exists='replace', 
                    index=False, 
                    chunksize=20000 # Bloques de 20k registros
                )
                conn.commit() # Aseguramos que se guarde
                
            print(f" ----> {len(df)} filas subidas correctamente.")

        except Exception as e:
            print(f"Error crítico: {e}")
            #considera liberar por si falla 
            engine.dispose()

cargar_multiples_tablas_seguro(tb_data, engine)

🚀 Iniciando carga optimizada de tb_facturacion...
 ----> 5000 filas subidas correctamente.


In [ ]:
engine.dispose() 